In [ ]:
%pip install --upgrade pip
%pip install -r requirements.txt

In [ ]:
%pip install ruamel.yaml

In [ ]:
import boto3

from ruamel.yaml import YAML
from io import StringIO

In [ ]:
yaml = YAML()
with open('pre-requisite.yaml', 'r') as file:
    yaml_data = yaml.load(file)

In [ ]:
cloudformation_client = boto3.client('cloudformation')
stack_name = 'kb-observability-stack'

In [ ]:
stream = StringIO()
yaml.dump(yaml_data, stream)

template_body = stream.getvalue()

In [ ]:
response = cloudformation_client.create_stack(
            StackName = stack_name,
            TemplateBody = template_body,
            Capabilities=['CAPABILITY_IAM'])

In [ ]:
cloudformation_client.get_waiter('stack_create_complete').wait(StackName=stack_name)

In [ ]:
response = cloudformation_client.describe_stacks(StackName=stack_name)
output_dict = {}

for output in response['Stacks'][0]['Outputs']:
    output_dict[output['OutputKey']] = output['OutputValue']

In [ ]:
output_dict_select = {"FIREHOSE_NAME": output_dict["FirehoseDeliveryStreamName"], 
                      "CRAWLER_NAME": output_dict["GlueCrawlerName"]}

In [ ]:
session = boto3.Session()
region = session.region_name

In [ ]:
output_dict_select['REGION'] = region

In [ ]:
with open('config.py', 'w') as configfile:
    for key, value in output_dict_select.items():
        configfile.write(f"{key} = '{value}'\n")